In [3]:
from bs4 import BeautifulSoup
from pandas import ExcelWriter
from pandas import ExcelFile
from html.parser import HTMLParser
from html.entities import name2codepoint

In [4]:
import requests
import pandas as pd
import os
import datetime

In [5]:
#Valor da hora para utilizacao nas pesquisas
now = datetime.datetime.now()

In [6]:
#Variáveis utilizadas
volume = 50000

In [7]:
#Verifica diretório Atual 
os.getcwd() 

'C:\\Users\\filip\\Documents\\Python Scripts'

In [8]:
#Configura Diretório
os.chdir('C:\\Users\\filip\\Documents\\Python Scripts')

In [133]:
#Importa tabela de Ticker que deve estar no diretório de Trabalho
index_table = pd.read_excel('TICKERS.xlsx', sheet_name='Sheet1')

index_table['STATUS'] = 'PENDENTE'

#Verifica a importação
#index_table.describe()
index_table.head(5)

,TICKER,NOME,DESCRICAO,STATUS
0,AALR3,ALLIAR,CENTRO DE IMAGEM DIAGNOSTICOS S.A.,PENDENTE
1,ABCB3,ABC Brasil,BANCO ABC BRASIL S/A,PENDENTE
2,ABCB4,ABC Brasil,BANCO ABC BRASIL S/A,PENDENTE
3,ABEV3,AMBEV S/A,AMBEV S.A.,PENDENTE
4,ABRE3,SOMOS EDUCA,SOMOS EDUCAÇÃO S.A.,PENDENTE


In [12]:
#Importa direto a planilha caso já esteja preenchida. Geralmente comentar esta linha
tabela_ord_rank = pd.read_excel("dados_balanco_calculado_2019-04-19.xlsx", sheet_name='Sheet1')
tabela_ord_rank.head(5)

,TICKER,NOME_EMPRESA,TIPO,SETOR,SUBSETOR,DATA_ULT_BAL,NUM_ACOES,VOL_MEDIO,COTACAO,VALOR_MERCADO,...,PATRIM_LIQUIDO,REC_LIQ_ANU,LUCRO_LIQ_ANU,EBIT_ANU,REC_LIQ_TRI,EBIT_TRI,LUCRO_LIQ_TRI,ORD_EVEBIT,ORD_ROIC,RANKING
882,WIZS3,WIZ S.A. ON NM,ON NM,Previdência e Seguros,Corretoras de Seguros,31/12/2018,159907000,5184380,8.46,1352810000,...,168973000,585005000,177726000,329296000,154168000,88260000,48574000,3,2,5
630,PSSA3,PORTO SEGURO SA ON,ON,Previdência e Seguros,Seguradoras,31/12/2018,323293000,39941300,55.32,17884600000,...,7643240000,17822000000,1310620000,14485500000,4619620000,3695860000,385050000,1,4,5
841,UNIP6,UNIPAR PARTICIPAÇÕES S.A. PNB,PNB,Químicos,Petroquímicos,31/12/2018,87155000,7183610,39.19,3415600000,...,1347520000,3469130000,528040000,861197000,922818000,172625000,153808000,4,11,15
839,UNIP3,UNIPAR PARTICIPAÇÕES S.A. ON,ON,Químicos,Petroquímicos,31/12/2018,87155000,307639,39.76,3465280000,...,1347520000,3469130000,528040000,861197000,922818000,172625000,153808000,5,12,17
252,CRPG6,CRISTAL PNB,PNB,Químicos,Químicos Diversos,31/12/2018,23215000,65892,30.50,708057000,...,509054000,636045000,155672000,150777000,177747000,41107000,45681000,7,19,26


In [134]:
#Cria tabela para guardar indicadores das Empresas
tabela_inicial = pd.DataFrame([],columns = ['TICKER','NOME_EMPRESA','TIPO','SETOR','SUBSETOR','DATA_ULT_BAL','NUM_ACOES','VOL_MEDIO','COTACAO','VALOR_MERCADO','VALOR_FIRMA','P/L','P/VP','PSR','DIV_YIELD','P/EBIT','CRES_REC_5A','LPA','VPA','MARG_BRUTA','MARG_EBIT','MARG_LIQUIDA','EBIT_ATIVO','ROIC','ROE','EV/EBIT','LIQUIDEZ_CORR','DIV_BR_PATRIM','ATIVO', 'DISPONIBILIDADES', 'ATIVO_CIRCULANTE', 'DIV_BRUTA', 'DIV_LIQUIDA', 'PATRIM_LIQUIDO','REC_LIQ_ANU','LUCRO_LIQ_ANU','EBIT_ANU','REC_LIQ_TRI','EBIT_TRI','LUCRO_LIQ_TRI'])
tabela_inicial

,TICKER,NOME_EMPRESA,TIPO,SETOR,SUBSETOR,DATA_ULT_BAL,NUM_ACOES,VOL_MEDIO,COTACAO,VALOR_MERCADO,...,ATIVO_CIRCULANTE,DIV_BRUTA,DIV_LIQUIDA,PATRIM_LIQUIDO,REC_LIQ_ANU,LUCRO_LIQ_ANU,EBIT_ANU,REC_LIQ_TRI,EBIT_TRI,LUCRO_LIQ_TRI


In [135]:
#Classe para buscar as informações de text das tags HTML
class MyHTMLParserData(HTMLParser):
    def handle_data(self, data):
        self.valor = data

In [136]:
#Instancia Classe criada acima
parserData = MyHTMLParserData()

In [137]:
#Classe para tratamento do registro e inclusão dos dados na tabela tabela_inicial
class finance_table():
    def init_table(self, ticker, row_number):
        tabela_inicial.loc[row_number] = [ticker,'','','','',0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        
    def update_table(self, row_number, column_name, column_value):
        if column_name == 'STATUS' or column_name == 'TICKER' or column_name == 'NOME_EMPRESA' or column_name == 'TIPO' or column_name == 'SETOR' or column_name == 'SUBSETOR' or column_name == 'DATA_ULT_BAL':
            tabela_inicial.loc[[row_number],[column_name]] = column_value
        elif column_name == 'DATA_ULT_BAL':
            format_str = '%d/%m/%Y'
            datetime_obj = datetime.datetime.strptime(column_value, format_str)
            tabela_inicial.loc[[row_number],[column_name]] = datetime_obj.date()
        else:
            self.ajusted_value = column_value.replace(".", "")
            self.ajusted_value = self.ajusted_value.replace(" ", "")
            self.ajusted_value = self.ajusted_value.replace("\n", "")
            self.ajusted_value = self.ajusted_value.replace(",", ".")
            self.ajusted_value = self.ajusted_value.replace("%", "")
            
            if self.ajusted_value.find("-") != -1 and len(self.ajusted_value) == 1:
                self.ajusted_value = 0
            
            self.ajusted_float = float(self.ajusted_value)
            
            tabela_inicial.loc[[row_number],[column_name]] = self.ajusted_float

In [139]:
#Leitura das tags do site Fundamentus para cada empresa da tabela index_table (tickers)
index = 0
for index, row in index_table.iterrows():
    print("ticker: ",index_table.TICKER[index].lower(), index)
    
    if index_table.STATUS[index] == "PENDENTE":
        print("ATUALIZANDO") 
        r  = requests.get("http://www.fundamentus.com.br/detalhes.php?papel=" + index_table.TICKER[index].lower())
        data = r.text
        soup = BeautifulSoup(data)
        table = soup.find_all('table',class_='w728')

        lista = []
        row_td = []
        lista_pai = []
        i = 0
    
        for table_td in table:
            lista_pai = table_td.find_all('td')
            for row_td in lista_pai:
                lista.append(row_td)
            
        anual = 0        
        data_ult = 0
        ft = finance_table()
        ft.init_table(index_table.TICKER[index],index)
            
        for i in range(len(lista)):
        
            if str(lista[i]).find(">Últ balanço processado<")>0 and data_ult == 0: 
                parserData.feed(str(lista[i+1]))
                #print("Últ balanço processado: ",parserData.valor)
                ft.update_table(index,'DATA_ULT_BAL',parserData.valor)
                data_ult = 1
            
            elif str(lista[i]).find(">Cotação<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Cotação: ",parserData.valor)
                ft.update_table(index,'COTACAO',parserData.valor)
            
            elif str(lista[i]).find(">Tipo<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Tipo: ",parserData.valor)
                ft.update_table(index,'TIPO',parserData.valor)
            
            elif str(lista[i]).find(">Empresa<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Empresa: ",parserData.valor)
                ft.update_table(index,'NOME_EMPRESA',parserData.valor)
            
            elif str(lista[i]).find(">Setor<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Setor: ",parserData.valor)
                ft.update_table(index,'SETOR',parserData.valor)
            
            elif str(lista[i]).find(">Subsetor<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Subsetor: ",parserData.valor)
                ft.update_table(index,'SUBSETOR',parserData.valor)
            
            elif str(lista[i]).find(">Últ balanço processado<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Últ balanço processado: ",parserData.valor)
                ft.update_table(index,'DATA_ULT_BAL',parserData.valor)
            
            elif str(lista[i]).find(">Vol $ méd (2m)<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Vol $ méd (2m): ",parserData.valor)
                ft.update_table(index,'VOL_MEDIO',parserData.valor)

            elif str(lista[i]).find(">Valor de mercado<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Valor de mercado: ",parserData.valor)
                ft.update_table(index,'VALOR_MERCADO',parserData.valor)

            elif str(lista[i]).find(">Valor da firma<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Valor da firma: ",parserData.valor)
                ft.update_table(index,'VALOR_FIRMA',parserData.valor)            

            elif str(lista[i]).find(">Nro. Ações<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Nro. Ações: ",parserData.valor)
                ft.update_table(index,'NUM_ACOES',parserData.valor)

            elif str(lista[i]).find(">P/L<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("P/L: ",parserData.valor)
                ft.update_table(index,'P/L',parserData.valor)

            elif str(lista[i]).find(">LPA<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("LPA: ",parserData.valor)
                ft.update_table(index,'LPA',parserData.valor)

            elif str(lista[i]).find(">P/VP<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("P/VP: ",parserData.valor)
                ft.update_table(index,'P/VP',parserData.valor)

            elif str(lista[i]).find(">VPA<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("VPA: ",parserData.valor)
                ft.update_table(index,'VPA',parserData.valor)

            elif str(lista[i]).find(">P/EBIT<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("P/EBIT: ",parserData.valor)
                ft.update_table(index,'P/EBIT',parserData.valor)
            
            elif str(lista[i]).find(">Marg. Bruta<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Marg. Bruta: ",parserData.valor)
                ft.update_table(index,'MARG_BRUTA',parserData.valor) 
            
            elif str(lista[i]).find(">PSR<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("PSR: ",parserData.valor)
                ft.update_table(index,'PSR',parserData.valor) 
            
            elif str(lista[i]).find(">Marg. EBIT<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Marg. EBIT: ",parserData.valor)
                ft.update_table(index,'MARG_EBIT',parserData.valor) 
            
            elif str(lista[i]).find(">Marg. Líquida<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Marg. Líquida: ",parserData.valor)
                ft.update_table(index,'MARG_LIQUIDA',parserData.valor) 
            
            elif str(lista[i]).find(">EBIT / Ativo<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("EBIT / Ativo: ",parserData.valor)
                ft.update_table(index,'EBIT_ATIVO',parserData.valor) 
            
            elif str(lista[i]).find(">ROIC<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("ROIC: ",parserData.valor)
                ft.update_table(index,'ROIC',parserData.valor)           
            
            elif str(lista[i]).find(">Div. Yield<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Div. Yield: ",parserData.valor)
                ft.update_table(index,'DIV_YIELD',parserData.valor) 
            
            elif str(lista[i]).find(">ROE<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("ROE: ",parserData.valor)
                ft.update_table(index,'ROE',parserData.valor) 
            
            elif str(lista[i]).find(">EV / EBIT<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("EV / EBIT: ",parserData.valor)
                ft.update_table(index,'EV/EBIT',parserData.valor) 
            
            elif str(lista[i]).find(">Liquidez Corr<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Liquidez Corr: ",parserData.valor)
                ft.update_table(index,'LIQUIDEZ_CORR',parserData.valor)   
            
            elif str(lista[i]).find(">Div Br/ Patrim<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Div Br/ Patrim: ",parserData.valor)
                ft.update_table(index,'DIV_BR_PATRIM',parserData.valor)  
            
            elif str(lista[i]).find(">Cres. Rec (5a)<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Cres. Rec (5a): ",parserData.valor)
                ft.update_table(index,'CRES_REC_5A',parserData.valor)  
                        
            elif str(lista[i]).find(">Ativo<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Ativo: ",parserData.valor)
                ft.update_table(index,'ATIVO',parserData.valor)  
            
            elif str(lista[i]).find(">Disponibilidades<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Disponibilidades: ",parserData.valor)
                ft.update_table(index,'DISPONIBILIDADES',parserData.valor)  
            
            elif str(lista[i]).find(">Ativo Circulante<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Ativo Circulante: ",parserData.valor)
                ft.update_table(index,'ATIVO_CIRCULANTE',parserData.valor)    
            
            elif str(lista[i]).find(">Dív. Bruta<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Dív. Bruta: ",parserData.valor)
                ft.update_table(index,'DIV_BRUTA',parserData.valor)  
            
            elif str(lista[i]).find(">Dív. Líquida<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Dív. Líquida: ",parserData.valor)
                ft.update_table(index,'DIV_LIQUIDA',parserData.valor)  
            
            elif str(lista[i]).find(">Patrim. Líq<")>0: 
                parserData.feed(str(lista[i+1]))
                #print("Patrim. Líq: ",parserData.valor)
                ft.update_table(index,'PATRIM_LIQUIDO',parserData.valor)              
            
            elif str(lista[i]).find(">Receita Líquida<")>0:
                parserData.feed(str(lista[i+1]))
                if anual != 1:
                    anual = 1
                    #print("Receita Líquida Anual: ",parserData.valor)
                    ft.update_table(index,'REC_LIQ_ANU',parserData.valor)
                else:
                    #print("Receita líquida Trimestral: ",parserData.valor)
                    ft.update_table(index,'REC_LIQ_TRI',parserData.valor)
                    anual = 0
                
            elif str(lista[i]).find(">EBIT<")>0:
                parserData.feed(str(lista[i+1]))
                if anual != 1:
                    anual = 1
                    #print("EBIT Anual: ",parserData.valor)
                    ft.update_table(index,'EBIT_ANU',parserData.valor)            
                else:
                    #print("EBIT Trimestral: ",parserData.valor)
                    ft.update_table(index,'EBIT_TRI',parserData.valor)        
                    anual = 0 
                    
            elif str(lista[i]).find(">Lucro Líquido<")>0:
                parserData.feed(str(lista[i+1]))
                if anual != 1:
                    anual = 1
                    #print("Lucro Líquido Anual: ",parserData.valor)
                    ft.update_table(index,'LUCRO_LIQ_ANU',parserData.valor)            
                else:
                    #print("Lucro Líquido Trimestral: ",parserData.valor)
                    ft.update_table(index,'LUCRO_LIQ_TRI',parserData.valor)        
                    anual = 0
                    
        #Atualiza Status                
        ft.update_table(index,'STATUS','ATUALIZADO')

ticker:  aalr3 0
ATUALIZANDO
ticker:  abcb3 1
ATUALIZANDO
ticker:  abcb4 2
ATUALIZANDO
ticker:  abev3 3
ATUALIZANDO
ticker:  abre3 4
ATUALIZANDO
ticker:  abya3 5
ATUALIZANDO
ticker:  aces3 6
ATUALIZANDO
ticker:  aces4 7
ATUALIZANDO
ticker:  adhm3 8
ATUALIZANDO
ticker:  aedu11 9
ATUALIZANDO
ticker:  aedu3 10
ATUALIZANDO
ticker:  aelp3 11
ATUALIZANDO
ticker:  aesl3 12
ATUALIZANDO
ticker:  aesl4 13
ATUALIZANDO
ticker:  aflt3 14
ATUALIZANDO
ticker:  aflu3 15
ATUALIZANDO
ticker:  aflu5 16
ATUALIZANDO
ticker:  agei3 17
ATUALIZANDO
ticker:  agen33 18
ATUALIZANDO
ticker:  agin3 19
ATUALIZANDO
ticker:  agro3 20
ATUALIZANDO
ticker:  aheb3 21
ATUALIZANDO
ticker:  aheb5 22
ATUALIZANDO
ticker:  aheb6 23
ATUALIZANDO
ticker:  alba3 24
ATUALIZANDO
ticker:  alll11 25
ATUALIZANDO
ticker:  alll3 26
ATUALIZANDO
ticker:  alll4 27
ATUALIZANDO
ticker:  alpa3 28
ATUALIZANDO
ticker:  alpa4 29
ATUALIZANDO
ticker:  alsc3 30
ATUALIZANDO
ticker:  alup11 31
ATUALIZANDO
ticker:  alup3 32
ATUALIZANDO
ticker:  alup4 3

ticker:  ctip3 268
ATUALIZANDO
ticker:  ctka3 269
ATUALIZANDO
ticker:  ctka4 270
ATUALIZANDO
ticker:  ctnm3 271
ATUALIZANDO
ticker:  ctnm4 272
ATUALIZANDO
ticker:  ctpc3 273
ATUALIZANDO
ticker:  ctpc4 274
ATUALIZANDO
ticker:  ctsa3 275
ATUALIZANDO
ticker:  ctsa4 276
ATUALIZANDO
ticker:  ctsa8 277
ATUALIZANDO
ticker:  ctwr3 278
ATUALIZANDO
ticker:  cvcb3 279
ATUALIZANDO
ticker:  cyre3 280
ATUALIZANDO
ticker:  cyre4 281
ATUALIZANDO
ticker:  czlt33 282
ATUALIZANDO
ticker:  czrs3 283
ATUALIZANDO
ticker:  czrs4 284
ATUALIZANDO
ticker:  dagb33 285
ATUALIZANDO
ticker:  dasa3 286
ATUALIZANDO
ticker:  dayc3 287
ATUALIZANDO
ticker:  dayc4 288
ATUALIZANDO
ticker:  dfva3 289
ATUALIZANDO
ticker:  dfva4 290
ATUALIZANDO
ticker:  dhbi3 291
ATUALIZANDO
ticker:  dhbi4 292
ATUALIZANDO
ticker:  dirr3 293
ATUALIZANDO
ticker:  djon4 294
ATUALIZANDO
ticker:  dmmo3 295
ATUALIZANDO
ticker:  doca3 296
ATUALIZANDO
ticker:  doca4 297
ATUALIZANDO
ticker:  dohl3 298
ATUALIZANDO
ticker:  dohl4 299
ATUALIZANDO
ticker

ticker:  mlft4 532
ATUALIZANDO
ticker:  mlpa12 533
ATUALIZANDO
ticker:  mlpa3 534
ATUALIZANDO
ticker:  mlpa4 535
ATUALIZANDO
ticker:  mmaq3 536
ATUALIZANDO
ticker:  mmaq4 537
ATUALIZANDO
ticker:  mmxm3 538
ATUALIZANDO
ticker:  mndl3 539
ATUALIZANDO
ticker:  mndl4 540
ATUALIZANDO
ticker:  mnpr3 541
ATUALIZANDO
ticker:  mnpr4 542
ATUALIZANDO
ticker:  mnsa3 543
ATUALIZANDO
ticker:  mnsa4 544
ATUALIZANDO
ticker:  moar3 545
ATUALIZANDO
ticker:  movi3 546
ATUALIZANDO
ticker:  mplu3 547
ATUALIZANDO
ticker:  mrfg3 548
ATUALIZANDO
ticker:  mrsl3 549
ATUALIZANDO
ticker:  mrsl4 550
ATUALIZANDO
ticker:  mrve3 551
ATUALIZANDO
ticker:  msan3 552
ATUALIZANDO
ticker:  msan4 553
ATUALIZANDO
ticker:  mspa3 554
ATUALIZANDO
ticker:  mspa4 555
ATUALIZANDO
ticker:  mtbr3 556
ATUALIZANDO
ticker:  mtbr4 557
ATUALIZANDO
ticker:  mtig3 558
ATUALIZANDO
ticker:  mtig4 559
ATUALIZANDO
ticker:  mtsa4 560
ATUALIZANDO
ticker:  mult3 561
ATUALIZANDO
ticker:  mwet3 562
ATUALIZANDO
ticker:  mwet4 563
ATUALIZANDO
ticker:

ticker:  tmcp3 796
ATUALIZANDO
ticker:  tmcp4 797
ATUALIZANDO
ticker:  tmgc11 798
ATUALIZANDO
ticker:  tmgc12 799
ATUALIZANDO
ticker:  tmgc13 800
ATUALIZANDO
ticker:  tmgc3 801
ATUALIZANDO
ticker:  tmgc7 802
ATUALIZANDO
ticker:  tncp3 803
ATUALIZANDO
ticker:  tncp4 804
ATUALIZANDO
ticker:  tnep3 805
ATUALIZANDO
ticker:  tnep4 806
ATUALIZANDO
ticker:  tnlp3 807
ATUALIZANDO
ticker:  tnlp4 808
ATUALIZANDO
ticker:  tots3 809
ATUALIZANDO
ticker:  toyb3 810
ATUALIZANDO
ticker:  toyb4 811
ATUALIZANDO
ticker:  tpis3 812
ATUALIZANDO
ticker:  tprc3 813
ATUALIZANDO
ticker:  tprc6 814
ATUALIZANDO
ticker:  trfo3 815
ATUALIZANDO
ticker:  trfo4 816
ATUALIZANDO
ticker:  tris3 817
ATUALIZANDO
ticker:  tror3 818
ATUALIZANDO
ticker:  tror4 819
ATUALIZANDO
ticker:  trpl3 820
ATUALIZANDO
ticker:  trpl4 821
ATUALIZANDO
ticker:  trpn3 822
ATUALIZANDO
ticker:  tsep3 823
ATUALIZANDO
ticker:  tsep4 824
ATUALIZANDO
ticker:  tspp3 825
ATUALIZANDO
ticker:  tspp4 826
ATUALIZANDO
ticker:  tupy3 827
ATUALIZANDO
ticke

In [1]:
#Verifica a inclusão dos registros
tabela_inicial.head(5)

NameError: name 'tabela_inicial' is not defined

In [141]:
#Retira Valores zerados para os indicadores que iremos utilizar
tabela_szero = tabela_inicial.query("VALOR_FIRMA > 0 and ROIC > 0 and EBIT_ANU > 0")

In [142]:
#Volume maior do que 50000
tabela_volume = tabela_szero[tabela_szero['VOL_MEDIO'] > volume]

In [143]:
#Filtra somente balanços do ano atual e passado para não pegar sujeira como empresas que não estão mais na bolsa
tabela_year = tabela_volume[(tabela_volume['DATA_ULT_BAL'].str.contains(str(now.year-1)))|(tabela_volume['DATA_ULT_BAL'].str.contains( str(now.year)))]

In [144]:
#Ordenando por EV/EBIT
tabela_ord1 = tabela_year.sort_values('EV/EBIT',ascending=True)

In [145]:
#Rankeando EV/EBIT
index = 0
count = 0
for index, row in tabela_ord1.iterrows():
    count = count + 1
    tabela_ord1.loc[index,'ORD_EVEBIT'] = count

In [146]:
#Ordenando por ROIC
tabela_ord2 = tabela_ord1.sort_values('ROIC',ascending=False)

In [147]:
#Rankeando ROIC
index = 0
count = 0
for index, row in tabela_ord2.iterrows():
    count = count + 1
    tabela_ord2.loc[index,'ORD_ROIC'] = count

In [148]:
#Somando os 2 rankeamentos
index = 0
count = 0
for index, row in tabela_ord2.iterrows():
    count = count + 1
    tabela_ord2.loc[index,'RANKING'] = tabela_ord2.loc[index,'ORD_EVEBIT'] + tabela_ord2.loc[index,'ORD_ROIC']

In [149]:
#Ordena tabela de Rankeamento
tabela_ord_rank = tabela_ord2.sort_values('RANKING',ascending=False)

In [150]:
#Exporta o Excel com a formula mágica
tabela_ord_rank.to_excel("dados_balanco_calculado_"+now.strftime("%Y-%m-%d")+".xlsx")

In [13]:
tabela_ord_rank.head(5)

,TICKER,NOME_EMPRESA,TIPO,SETOR,SUBSETOR,DATA_ULT_BAL,NUM_ACOES,VOL_MEDIO,COTACAO,VALOR_MERCADO,...,PATRIM_LIQUIDO,REC_LIQ_ANU,LUCRO_LIQ_ANU,EBIT_ANU,REC_LIQ_TRI,EBIT_TRI,LUCRO_LIQ_TRI,ORD_EVEBIT,ORD_ROIC,RANKING
882,WIZS3,WIZ S.A. ON NM,ON NM,Previdência e Seguros,Corretoras de Seguros,31/12/2018,159907000,5184380,8.46,1352810000,...,168973000,585005000,177726000,329296000,154168000,88260000,48574000,3,2,5
630,PSSA3,PORTO SEGURO SA ON,ON,Previdência e Seguros,Seguradoras,31/12/2018,323293000,39941300,55.32,17884600000,...,7643240000,17822000000,1310620000,14485500000,4619620000,3695860000,385050000,1,4,5
841,UNIP6,UNIPAR PARTICIPAÇÕES S.A. PNB,PNB,Químicos,Petroquímicos,31/12/2018,87155000,7183610,39.19,3415600000,...,1347520000,3469130000,528040000,861197000,922818000,172625000,153808000,4,11,15
839,UNIP3,UNIPAR PARTICIPAÇÕES S.A. ON,ON,Químicos,Petroquímicos,31/12/2018,87155000,307639,39.76,3465280000,...,1347520000,3469130000,528040000,861197000,922818000,172625000,153808000,5,12,17
252,CRPG6,CRISTAL PNB,PNB,Químicos,Químicos Diversos,31/12/2018,23215000,65892,30.50,708057000,...,509054000,636045000,155672000,150777000,177747000,41107000,45681000,7,19,26
